In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1><center>Lost at space?
    </center></h1>


<p align="center">
  <img width="2000" height="1000" src="https://cdn.pixabay.com/photo/2017/08/30/01/05/milky-way-2695569__480.jpg">
</p>


## 1. Feature Observation (Visualization)
* Dividing the columns according to their data types 
* Using Seaborn for visualization

## 2. Feature Engineering
* Extracting the Cabin column
* Perform one hot encoding for necessary features
* Getting features ready for our model (train + validation split)

## 3. Training + Validation
* Random Forest Model Implementation
* XGBoost Model Implementation
* Hyperparameter Tuning through Random Search

## 4. Machine Learning Model Report (Random Forest)
* Confusion Matrix
* Feature Importances

## 5. Summary
* Making our submission file
* Possible areas of improvement
* acknowledgements

## 6. PyTorch

In [ ]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')
sample_df = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

# 1. Feature Observation

In [ ]:
train_df

#### Let us first divide the features into 'objects', 'boolean', 'float' datatypes

In [ ]:
objects = train_df.select_dtypes(include='object')
boolean = train_df.select_dtypes(include='bool')
numbers = train_df.select_dtypes(include='float')

## 1.1 Numerical Feature Observation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,20))
    
# Draw the plot
for count, feature in enumerate(numbers.columns.tolist()):
    ax = plt.subplot(3, 2, count+1)    
    ax.set_title("{}".format(feature), size = 30)
    ax.hist(numbers[feature],color = 'pink', edgecolor = 'black', bins = 50)
    
plt.suptitle("Distribution of Numerical Features", fontsize = 50)

> #### Seems like majority of the people onboard did not spend much at each of the Spaceship Titanic's many luxury amenities. 
> #### For the age wise, it seems like majority of our passengers were between the ages of 20 to 30 while there was a pretty large number of infants onboard too

## 1.2 Object Feature Observation

In [ ]:
objects

In [ ]:
for i in objects.columns.tolist()[1:-1]:
    print(objects[i].describe(),"\n")

In [ ]:
sns.set_theme(style="darkgrid")
fig, ax =plt.subplots(2, 2,figsize=(15, 15))
sns.countplot(x = 'HomePlanet', data = objects, ax=ax[0][0])
sns.countplot(x = 'CryoSleep', data=objects, ax=ax[0][1])
sns.countplot(x = 'Destination', data = objects, ax=ax[1][0])
sns.countplot(x = 'VIP', data = objects, ax=ax[1][1])
fig.suptitle('Comparisons of different Categorical Features', fontsize=20)
fig.show()

> #### Seems like most of the people on board were from Earth and a vast majority of them were headed for the TRAPPIST-1e. 
> #### It can also be seen that almost all of them were not VIP and about 5500 of them did not opt for cryosleep for the journey
> #### For Cabin, we will leave that to the feature engineering section to accomplish and we wound visualize the outcomes then

# 2. Feature Engineering

#### I will first engineer the **Cabin** column which seems pretty important
#### The column contains these following information
* deck
* num
* side

#### If we can extract out those informations to three different columns, our model would be able to extract weights from those individual columns for a possibly more accurate prediction

In [ ]:
objects['Cabin']

In [ ]:
objects['Cabin'].fillna(0, inplace=True)

In [ ]:
import math
deck = []
num = []
side = []

for i in range(len(objects['Cabin'])):
    if objects['Cabin'].iloc[i] == 0:
        result = [math.nan, math.nan, math.nan]
    else:
        result = objects['Cabin'].iloc[i].split('/')
    deck.append(result[0])
    num.append(result[1])
    side.append(result[2])
    
objects['Deck'] = deck
objects['Num'] = num
objects['Side'] = side

objects

In [ ]:
sns.set_theme(style="darkgrid")
fig, ax =plt.subplots(1,2,figsize=(15, 15))
sns.countplot(x = 'Deck', data = objects, ax=ax[0])
sns.countplot(x = 'Side', data=objects, ax=ax[1])

fig.suptitle('Comparisons of elements of Cabin feature', fontsize=20)
fig.show()

In [ ]:
#Changing datatype of 'Num' into float for visualization as shown below
objects = objects.astype({"Num": float})

In [ ]:
sns.displot(objects, x="Num",kind="kde", height=10)

> #### It seems like for the Decks, majority of the people were in F and G while the number on the portside was similar to the ones on the starboard side. 
> #### For the Num column, seems like the majority is around 100 to 200. This Cabin number at first might seem like it doesn't pose much information but since we do not know the layout of the spaceship, leaving the learning to the model might help the model predict the room number in which the occupants had a higher rate of survival possibility, probably due to the distance to the exit

#### Now let us ensure our feature columns are in one single dataframe and split them into 70% training data and 30% validation data

## 2.1 Data Preparation

In [ ]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')

train_df['label'] = 'train'
test_df['label'] = 'test'

# Concat
concat_df = pd.concat([train_df , test_df])

In [ ]:
#Extracting out the 'Cabin' column
concat_df['Cabin'].fillna(0, inplace=True)

deck = []
num = []
side = []

for i in range(len(concat_df['Cabin'])):
    if concat_df['Cabin'].iloc[i] == 0:
        result = [math.nan, math.nan, math.nan]
    else:
        result = concat_df['Cabin'].iloc[i].split('/')
    deck.append(result[0])
    num.append(result[1])
    side.append(result[2])
    
concat_df['Deck'] = deck
concat_df['Num'] = num
concat_df['Side'] = side

In [ ]:
concat_df.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)

> #### Let's proceed on with one-hot encoding for our categorical features

In [ ]:
onehotencoding = pd.get_dummies(concat_df.select_dtypes(include='object').drop(['Transported', 'label'], axis=1))
concat_df.drop(concat_df.select_dtypes(include='object').drop(['Transported', 'label'], axis=1), axis=1, inplace=True)
concat_set = pd.concat([concat_df, onehotencoding], axis=1)

# Split your data
train_df = concat_set[concat_set['label'] == 'train']
test_df = concat_set[concat_set['label'] == 'test']

# Drop your labels
train_df = train_df.drop('label', axis=1)
test_df = test_df.drop('label', axis=1)

> #### One hot encoding is done
> #### We won't carry out any feature scaling and normalization as we would be using Random Forest model for this classification problem
> #### However, we would need to fill in the missing (NaN) values for numerical columns for this model to work

In [ ]:
train_df.isna().sum().head(10)

#### For 'Age', we would use MissForest Imputation.

#### For the rest of the missing numerical columns, I will just impute it with the mode value as the data is very skewed towards the low values. 

In [ ]:
pip install missingpy

In [ ]:
from missingpy import MissForest

# Impute
imputer = MissForest()
data_imputed = imputer.fit_transform(train_df.iloc[: , :6])
data_imputed = pd.DataFrame(data=data_imputed, columns=train_df.iloc[: , :6].columns)

data_imputed

In [ ]:
train_df.drop(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], axis = 1, inplace=True)
train_df = pd.concat([train_df, data_imputed], axis=1)
train_df.isna().sum()

In [ ]:
train_df.Transported = train_df.Transported.astype(bool)

#### All the work is done for imputation 

In [ ]:
from sklearn.model_selection import train_test_split
y = train_df['Transported']
X = train_df.drop('Transported', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

# 3. Training + Validation

![](https://i.ytimg.com/vi/goPiwckWE9M/maxresdefault.jpg)

##### Image taken from https://www.youtube.com/watch?v=goPiwckWE9M

#### For this dataset, I would be using **Random Forest Classifier**. 
#### For those of you not familiar with this model, it is basically a ensemble method where there are numerous numbers of decision trees and averaging is used to improve the predictive accuracy and control over-fitting. 
#### This model heavily relies on the "Wisdom of Crowds"
#### As we know, decision trees can prove to be extremely accurate and does not need its features to be scaled nor normalized 
#### Furthermore, overfitting will be controlled later in the hyperparameter tuning section via hyperparameters such as n_estimators and max_depth
#### For more information, please visit this website https://towardsdatascience.com/understanding-random-forest-58381e0602d2 which I think does a terrific job of explaining this model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier(n_jobs=-1, random_state=42)

#Fitting the model
model.fit(X_train,y_train)

#Prediction
pred = model.predict(X_val)

#Evaluation
accuracy = accuracy_score(y_val, pred)
auc = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

#Statement
print("The accuracy is {} and roc_auc_score is {}".format(accuracy, auc))

#### Our model engineering won't be complete without some Hyperparameter Tuning which I will be performing using GridSearch with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [70,90, 110, 130],
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [2,4,6],
    'n_estimators': [100, 150, 200, 250, 300]
}
# Create a based model
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train,y_train)


#### So we successfully obtained our best performing parameters

In [ ]:
grid_search.best_params_

## Let us check the validation metrics with our improved model after some personal tuning too

In [ ]:
rf_model = RandomForestClassifier(n_jobs=-1, 
                               random_state=42, 
                               n_estimators = 230, 
                               min_samples_split = 5, 
                               min_samples_leaf = 1,  
                               max_depth = 110, 
                               bootstrap = True)

#Fitting the model
rf_model.fit(X_train,y_train)

#Prediction
pred = rf_model.predict(X_val)

#Evaluation
improved_accuracy = accuracy_score(y_val, pred)
improved_auc = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

improvement_in_accuracy = ((improved_accuracy-accuracy)/accuracy)*100
imporvement_in_auc = ((improved_auc-auc)/auc)*100

#Statement
print("After Hyperparameter Tuning, the accuracy is {} and roc_auc_score is {}".format(improved_accuracy, improved_auc))
print("Improvement in accuracy is {:.2f}% and in roc_auc_score is {:.2f}%".format(improvement_in_accuracy,imporvement_in_auc))

#### The tuned model has seen an improvement but not by a significant extent

#### For the second model, I will be using XGBoost. XGBoost is also an ensemble method but this algorithm is focused on limiting the error/loss that it's predictions bring

In [ ]:
from xgboost import XGBClassifier

boost_model = XGBClassifier(n_jobs=-1, random_state=42)

#Fitting the model
boost_model.fit(X_train,y_train)

#Prediction
pred = boost_model.predict(X_val)

#Evaluation
accuracy = accuracy_score(y_val, pred)
auc = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

#Statement
print("The accuracy is {} and roc_auc_score is {}".format(accuracy, auc))

#### It seems like our baseline Gradient Boosting model is not as accurate as our Random Forest Model. Let us try with some hyperparameters

In [ ]:
boost_model = XGBClassifier(n_jobs=-1, random_state=42, max_depth = 5)

#Fitting the model
boost_model.fit(X_train,y_train)

#Prediction
pred = boost_model.predict(X_val)

#Evaluation
improved_accuracy = accuracy_score(y_val, pred)
improved_auc = roc_auc_score(y_val, boost_model.predict_proba(X_val)[:, 1])

improvement_in_accuracy = ((improved_accuracy-accuracy)/accuracy)*100
imporvement_in_auc = ((improved_auc-auc)/auc)*100

#Statement
print("After Hyperparameter Tuning, the accuracy is {} and roc_auc_score is {}".format(improved_accuracy, improved_auc))
print("Improvement in accuracy is {:.2f}% and in roc_auc_score is {:.2f}%".format(improvement_in_accuracy,imporvement_in_auc))

#### Seems like our tuned Gradient Boosting Model has a very slight advantage over our Random Forest Model

# 4. Machine Learning Model Report (Random Forest)

#### Let us first look at our confusion matrix for an overview of our Random Forest model's performance

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_val, pred)

ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

sns.set(rc={'figure.figsize':(10,10)})
## Display the visualization of the Confusion Matrix.
plt.show()

#### Seems like 80% of our predictions lies in the True Positive and True Negative portion which seems like a decent number
#### The False Positive and False Negative proportions are also balanced

#### Now let us look at the feature importances which may give us a good insight as to which features were important and which were not as much

In [ ]:
feats = {} 
for feature, importance in zip(X.columns, model.feature_importances_):
    feats[feature] = importance
    
feature_importances = sorted(feats.items(), key=lambda x: x[1], reverse=True)

df = {"Feature": [], "Importance": []}
for i in feature_importances[:10]:
    df["Feature"].append(i[0])
    df['Importance'].append(i[1])
    
sns.set(rc={'figure.figsize':(20,15)})
sns.barplot(x='Feature', y='Importance', data = df)

#### The numerical columns seems to have had the highest importances. However, I have my doubts as I do not believe the amount spent at these amenities would greatly influence whether the person was transported. This may be due to several reasons. 
#### 1. The data for these numerical columns are very skewed to the lower values and most of the data points are 0
#### 2. Since values for these categories are very low, even a small change in these values might greatly influence our model's predictions (without much meaning), thus explaining the high feature importance placed by our model
#### The following features with the highest importances are whether the person opted for CryoSleep and their planet of Origin. 

# 5. Summary

#### Let us create a file to submit to the leaderboard

In [ ]:
# Impute
imputer = MissForest()
data_imputed = imputer.fit_transform(test_df.iloc[: , :6])
data_imputed = pd.DataFrame(data=data_imputed, columns=test_df.iloc[: , :6].columns)
test_df.drop(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], axis = 1, inplace=True)
test_df = pd.concat([test_df, data_imputed], axis=1)

test_df.drop('Transported', axis=1, inplace=True)

#submission
rf_submission = rf_model.predict(test_df)
boost_submission = boost_model.predict(test_df)

#submission file
sample_df['Transported'] = rf_submission
sample_df.to_csv('rf_submission.csv', index=False)

sample_df['Transported'] = boost_submission
sample_df.to_csv('boost_submission.csv', index=False)

In [ ]:
sample_df['Transported'] = boost_submission
sample_df.to_csv('submission.csv', index=False)

#### Our model seems to have performed decently with a rough accuracy of 79~80%. 
#### However, I do believe attempting other models to our data and selecting which feature to feed into our models might bring a higher accuracy. 
#### In the near future, I am planning on fitting **Gradient Boosting Model** (completed above) to our data and as to the feature engineering wise, I plan to try out different data imputation methods such as **KNNImputer**. Even different methods of hyperparameter tuning might help out, possibly using **Bayesian Optimization** which I am very interested in
#### So do check out my profile for more informative notebooks like this! 
#### Any suggestions and comments are very much welcomed! Thank you for reading :)

## Acknowledgements
### https://www.stackvidhya.com/plot-confusion-matrix-in-python-and-why/
### https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
### https://towardsdatascience.com/imputing-numerical-data-top-5-techniques-every-data-scientist-must-know-587c0f51552a